In [2]:
# Test Pareto Optimizer

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from robyn.data.entities.mmmdata import MMMData
from robyn.modeling.entities.modeloutputs import ModelOutputs, Trial
from robyn.modeling.pareto.pareto_optimizer import ParetoOptimizer
from robyn.data.entities.enums import DependentVarType, PaidMediaSigns, OrganicSigns, ContextSigns

from utils.data_mapper import export_data, import_data, save_data_to_json, load_data_from_json


In [3]:

# # 1. Create dummy data for MMMData

# # Generate date range
# date_range = pd.date_range(start='2022-01-01', end='2022-12-31', freq='D')

# # Create dummy data
# np.random.seed(42)
# data = pd.DataFrame({
#     'date': date_range,
#     'sales': np.random.randint(1000, 2000, size=len(date_range)),
#     'tv_spend': np.random.randint(100, 500, size=len(date_range)),
#     'radio_spend': np.random.randint(50, 200, size=len(date_range)),
#     'social_media_spend': np.random.randint(200, 600, size=len(date_range)),
#     'competitor_sales': np.random.randint(800, 1500, size=len(date_range)),
#     'holiday': np.random.choice([0, 1], size=len(date_range), p=[0.9, 0.1])
# })

# # Create MMMDataSpec
# mmmdata_spec = MMMData.MMMDataSpec(
#     dep_var='sales',
#     dep_var_type=DependentVarType.REVENUE,
#     date_var='date',
#     paid_media_spends=['tv_spend', 'radio_spend', 'social_media_spend'],
#     paid_media_vars=None,
#     paid_media_signs=[PaidMediaSigns.POSITIVE] * 3,
#     organic_vars=['competitor_sales'],
#     organic_signs=[OrganicSigns.NEGATIVE],
#     context_vars=['holiday'],
#     context_signs=[ContextSigns.POSITIVE],
#     factor_vars=None,
#     window_start=datetime(2022, 1, 1),
#     window_end=datetime(2022, 12, 31)
# )

# # Create MMMData instance
# mmm_data = MMMData(data, mmmdata_spec)


In [3]:

# # 2. Create dummy ModelOutputs

# def create_dummy_trial(trial_num):
#     num_solutions = 10  # Increase the number of solutions per trial
#     media_channels = ['tv_spend', 'radio_spend', 'social_media_spend', 'search_spend', 'print_spend']
    
#     result_hyp_param = pd.DataFrame({
#         'solID': [f'sol_{trial_num}_{i}' for i in range(num_solutions)],
#         'nrmse': np.random.uniform(0.1, 0.3, num_solutions),
#         'nrmse_train': np.random.uniform(0.1, 0.3, num_solutions),
#         'nrmse_test': np.random.uniform(0.1, 0.3, num_solutions),
#         'decomp.rssd': np.random.uniform(0.1, 0.3, num_solutions),
#         'mape': np.random.uniform(5, 15, num_solutions),
#         'rsq_train': np.random.uniform(0.7, 0.9, num_solutions),
#         'rsq_val': np.random.uniform(0.6, 0.8, num_solutions),
#         'rsq_test': np.random.uniform(0.5, 0.7, num_solutions),
#         'iterNG': [1] * num_solutions,
#         'iterPar': [trial_num] * num_solutions,
#         'lambda': np.random.uniform(0.01, 0.1, num_solutions),
#         'lambda_hp': np.random.uniform(0.01, 0.1, num_solutions),
#         'lambda_max': np.random.uniform(0.1, 0.5, num_solutions),
#         'lambda_min_ratio': np.random.uniform(0.01, 0.1, num_solutions)
#     })
    
#     # Ensure some solutions are Pareto-optimal
#     result_hyp_param['nrmse'] = np.sort(result_hyp_param['nrmse'])
#     result_hyp_param['decomp.rssd'] = np.sort(result_hyp_param['decomp.rssd'])[::-1]
    
#     x_decomp_agg = pd.DataFrame({
#         'solID': [f'sol_{trial_num}_{i}' for i in range(num_solutions) for _ in media_channels],
#         'rn': media_channels * num_solutions,
#         'coef': np.random.uniform(0.1, 0.5, num_solutions * len(media_channels)),
#         'boot_mean': np.random.uniform(0.1, 0.5, num_solutions * len(media_channels)),
#         'iterNG': [1] * (num_solutions * len(media_channels)),
#         'iterPar': [trial_num] * (num_solutions * len(media_channels))
#     })
    
#     decomp_spend_dist = pd.DataFrame({
#         'solID': [f'sol_{trial_num}_{i}' for i in range(num_solutions) for _ in media_channels],
#         'rn': media_channels * num_solutions,
#         'mean_spend': np.random.randint(100, 500, num_solutions * len(media_channels)),
#         'total_spend': np.random.randint(10000, 50000, num_solutions * len(media_channels)),
#         'xDecompAgg': np.random.randint(5000, 20000, num_solutions * len(media_channels)),
#         'spend_share': np.random.uniform(0, 1, num_solutions * len(media_channels)),
#         'effect_share': np.random.uniform(0, 1, num_solutions * len(media_channels)),
#         'iterNG': [1] * (num_solutions * len(media_channels)),
#         'iterPar': [trial_num] * (num_solutions * len(media_channels))
#     })
    
#     lift_calibration = pd.DataFrame({
#         'liftMedia': media_channels * num_solutions,
#         'lift': np.random.uniform(1, 2, num_solutions * len(media_channels)),
#         'iterNG': [1] * (num_solutions * len(media_channels)),
#         'iterPar': [trial_num] * (num_solutions * len(media_channels)),
#         'solID': [f'sol_{trial_num}_{i}' for i in range(num_solutions) for _ in media_channels]
#     }) if trial_num % 2 == 0 else None
    
#     return Trial(
#         result_hyp_param=result_hyp_param,
#         x_decomp_agg=x_decomp_agg,
#         decomp_spend_dist=decomp_spend_dist,
#         lift_calibration=lift_calibration,
#         nrmse=result_hyp_param['nrmse'].values[0],
#         decomp_rssd=result_hyp_param['decomp.rssd'].values[0],
#         mape=result_hyp_param['mape'].values[0],
#         rsq_train=result_hyp_param['rsq_train'].values[0],
#         rsq_val=result_hyp_param['rsq_val'].values[0],
#         rsq_test=result_hyp_param['rsq_test'].values[0],
#         lambda_=result_hyp_param['lambda'].values[0],
#         lambda_hp=result_hyp_param['lambda_hp'].values[0],
#         lambda_max=result_hyp_param['lambda_max'].values[0],
#         lambda_min_ratio=result_hyp_param['lambda_min_ratio'].values[0],
#         pos=trial_num,
#         elapsed=np.random.uniform(10, 30),
#         elapsed_accum=np.random.uniform(100, 300),
#         trial=trial_num,
#         iter_ng=1,
#         iter_par=trial_num,
#         train_size=0.7,
#         sol_id=f'sol_{trial_num}_0'
#     )

# # Create more trials
# trials = [create_dummy_trial(i) for i in range(1)]  # Increase the number of trials

# model_outputs = ModelOutputs(
#     trials=trials,
#     train_timestamp='2023-05-01 10:00:00',
#     cores=1,
#     iterations=10,
#     intercept=True,
#     intercept_sign='positive',
#     nevergrad_algo='NGOpt',
#     ts_validation=True,
#     add_penalty_factor=True,
#     hyper_updated={},
#     hyper_fixed=False,
#     convergence={},
#     select_id='sol_5',
#     seed=42,
#     hyper_bound_ng={},
#     hyper_bound_fixed={}
# )

In [2]:
# Load data from JSON
loaded_data = load_data_from_json(
    "/Users/funny/Downloads/exported_data.json"
)
imported_data = import_data(loaded_data)

mmm_data = imported_data["mmm_data"]
display(mmm_data.data.head())
# Display Model Outputs

model_outputs = imported_data["model_outputs"]
display((model_outputs.trials[0].result_hyp_param))
display((model_outputs.trials[0].decomp_spend_dist))
display((model_outputs.trials[0].x_decomp_agg))
display((model_outputs.trials[0].lift_calibration))

,DATE,revenue,tv_S,ooh_S,print_S,facebook_I,search_clicks_P,search_S,competitor_sales_B,facebook_S,events,newsletter
0,2015-11-23,2.754372e+06,22358.3467,0.0,12728.4889,2.430128e+07,0.0000,0.0000,8125009,7607.1329,na,19401.6538
1,2015-11-30,2.584277e+06,28613.4533,0.0,0.0000,5.527033e+06,9837.2385,4133.3333,7901549,1141.9525,na,14791.0000
2,2015-12-07,2.547387e+06,0.0000,132278.4,453.8667,1.665159e+07,12044.1197,3786.6667,8300197,4256.3754,na,14544.0000
3,2015-12-14,2.875220e+06,83450.3067,0.0,17680.0000,1.054977e+07,12268.0703,4253.3333,8122883,2800.4907,na,2800.0000
4,2015-12-21,2.215953e+06,0.0000,277336.0,0.0000,2.934090e+06,9467.2480,3613.3333,7105985,689.5826,na,15478.0000


,facebook_S_alphas,facebook_S_gammas,facebook_S_thetas,newsletter_alphas,newsletter_gammas,newsletter_thetas,ooh_S_alphas,ooh_S_gammas,ooh_S_thetas,print_S_alphas,...,lambda_hp,lambda_max,lambda_min_ratio,pos,Elapsed,ElapsedAccum,solID,trial,iterNG,iterPar
0,1.982029,0.584243,0.090727,1.751417,0.658512,0.228262,1.935087,0.804991,0.268471,1.347135,...,0.276251,6.548109e+07,0.0001,1,0.149130,0.149130,1_1_1,1,1,1
1,2.137653,0.423644,0.123685,1.721916,0.661215,0.278996,1.904210,0.597059,0.241335,1.468480,...,0.500997,6.548109e+07,0.0001,1,0.164648,0.168917,1_1_2,1,1,2
2,2.610267,0.796546,0.180935,1.474269,0.794208,0.218271,1.742035,0.579038,0.213408,2.664442,...,0.115435,6.548109e+07,0.0001,1,0.144982,0.153944,1_1_3,1,1,3
3,1.008753,0.477549,0.051473,0.972705,0.626001,0.249119,1.934297,0.545875,0.235703,1.432328,...,0.429544,6.548109e+07,0.0001,1,0.139280,0.152323,1_1_4,1,1,4
4,1.933208,0.629187,0.110968,1.857242,0.437809,0.199232,1.354886,0.744018,0.230233,1.667235,...,0.530087,6.548109e+07,0.0001,1,0.146084,0.164810,1_1_5,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002,2.465946,0.724777,0.225335,0.979106,0.499965,0.261373,0.522999,0.427181,0.356087,2.641617,...,0.000342,6.548109e+07,0.0001,1,0.091524,46.295074,1_223_5,1,223,5
2003,2.856181,0.873363,0.200188,0.633228,0.432423,0.384445,0.526098,0.806291,0.380355,2.674664,...,0.000518,6.548109e+07,0.0001,1,0.096245,46.305561,1_223_6,1,223,6
2004,0.675763,0.532316,0.173621,0.696997,0.974682,0.386295,0.512279,0.556015,0.390319,1.703621,...,0.000150,6.548109e+07,0.0001,1,0.087453,46.303110,1_223_7,1,223,7
2005,1.205384,0.753474,0.277343,0.970095,0.434559,0.266265,0.531863,0.752605,0.366073,2.615085,...,0.000387,6.548109e+07,0.0001,1,0.084403,46.305563,1_223_8,1,223,8


,rn,coef,xDecompAgg,xDecompPerc,xDecompMeanNon0,xDecompMeanNon0Perc,xDecompAggRF,xDecompPercRF,xDecompMeanNon0RF,xDecompMeanNon0PercRF,...,decomp.rssd,mape,lambda,lambda_hp,lambda_max,lambda_min_ratio,solID,trial,iterNG,iterPar
0,tv_S,85541.420821,2.170906e+06,0.007779,13827.429649,0.007756,2.170906e+06,0.007779,13827.429649,0.007756,...,0.738735,0,1.809395e+07,0.276251,6.548109e+07,0.0001,1_1_1,1,1,1
1,ooh_S,35948.624689,3.271705e+05,0.001172,2083.888774,0.001169,3.271705e+05,0.001172,2083.888774,0.001169,...,0.738735,0,1.809395e+07,0.276251,6.548109e+07,0.0001,1_1_1,1,1,1
2,print_S,37120.306797,9.594634e+05,0.003438,6111.231669,0.003428,9.594634e+05,0.003438,6111.231669,0.003428,...,0.738735,0,1.809395e+07,0.276251,6.548109e+07,0.0001,1_1_1,1,1,1
3,facebook_S,76517.641901,1.433262e+06,0.005136,9129.054591,0.005121,1.433262e+06,0.005136,9129.054591,0.005121,...,0.738735,0,1.809395e+07,0.276251,6.548109e+07,0.0001,1_1_1,1,1,1
4,search_S,127594.703964,3.809802e+06,0.013651,24266.252807,0.013612,3.809802e+06,0.013651,24266.252807,0.013612,...,0.738735,0,1.809395e+07,0.276251,6.548109e+07,0.0001,1_1_1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10030,tv_S,280475.696874,5.112657e+06,0.017694,32564.694947,0.015970,5.112657e+06,0.017694,32564.694947,0.015970,...,0.335769,0,4.155043e+04,0.000535,6.548109e+07,0.0001,1_223_9,1,223,9
10031,ooh_S,239185.189628,1.156273e+07,0.040016,73647.986054,0.036118,1.156273e+07,0.040016,73647.986054,0.036118,...,0.335769,0,4.155043e+04,0.000535,6.548109e+07,0.0001,1_223_9,1,223,9
10032,print_S,224429.812138,2.597955e+06,0.008991,16547.481450,0.008115,2.597955e+06,0.008991,16547.481450,0.008115,...,0.335769,0,4.155043e+04,0.000535,6.548109e+07,0.0001,1_223_9,1,223,9
10033,facebook_S,140706.741006,4.044137e+06,0.013996,25758.837154,0.012632,4.044137e+06,0.013996,25758.837154,0.012632,...,0.335769,0,4.155043e+04,0.000535,6.548109e+07,0.0001,1_223_9,1,223,9


,rn,coef,xDecompAgg,xDecompPerc,xDecompMeanNon0,xDecompMeanNon0Perc,xDecompAggRF,xDecompPercRF,xDecompMeanNon0RF,xDecompMeanNon0PercRF,...,decomp.rssd,mape,lambda,lambda_hp,lambda_max,lambda_min_ratio,solID,trial,iterNG,iterPar
0,(Intercept),1.490386e+06,2.339906e+08,0.838443,1.490386e+06,0.836009,2.339906e+08,0.838443,1.490386e+06,0.836009,...,0.738735,0,1.809395e+07,0.276251,6.548109e+07,0.0001,1_1_1,1,1,1
1,trend,8.528134e-02,2.386378e+07,0.085509,1.519986e+05,0.085261,2.386378e+07,0.085509,1.519986e+05,0.085261,...,0.738735,0,1.809395e+07,0.276251,6.548109e+07,0.0001,1_1_1,1,1,1
2,season,3.145892e-02,1.151530e+04,0.000041,7.334588e+01,0.000041,1.151530e+04,0.000041,7.334588e+01,0.000041,...,0.738735,0,1.809395e+07,0.276251,6.548109e+07,0.0001,1_1_1,1,1,1
3,holiday,2.209248e-02,1.538313e+05,0.000551,6.153252e+03,0.003452,1.538313e+05,0.000551,6.153252e+03,0.003452,...,0.738735,0,1.809395e+07,0.276251,6.548109e+07,0.0001,1_1_1,1,1,1
4,competitor_sales_B,1.061375e-02,9.247090e+06,0.033134,5.889866e+04,0.033038,9.247090e+06,0.033134,5.889866e+04,0.033038,...,0.738735,0,1.809395e+07,0.276251,6.548109e+07,0.0001,1_1_1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24079,ooh_S,2.391852e+05,1.156273e+07,0.040016,7.364799e+04,0.036118,1.156273e+07,0.040016,7.364799e+04,0.036118,...,0.335769,0,4.155043e+04,0.000535,6.548109e+07,0.0001,1_223_9,1,223,9
24080,print_S,2.244298e+05,2.597955e+06,0.008991,1.654748e+04,0.008115,2.597955e+06,0.008991,1.654748e+04,0.008115,...,0.335769,0,4.155043e+04,0.000535,6.548109e+07,0.0001,1_223_9,1,223,9
24081,facebook_S,1.407067e+05,4.044137e+06,0.013996,2.575884e+04,0.012632,4.044137e+06,0.013996,2.575884e+04,0.012632,...,0.335769,0,4.155043e+04,0.000535,6.548109e+07,0.0001,1_223_9,1,223,9
24082,search_S,2.411042e+05,6.712186e+06,0.023229,4.275278e+04,0.020966,6.712186e+06,0.023229,4.275278e+04,0.020966,...,0.335769,0,4.155043e+04,0.000535,6.548109e+07,0.0001,1_223_9,1,223,9


""


In [4]:

# 3. Create ParetoOptimizer instance
pareto_optimizer = ParetoOptimizer(mmm_data, model_outputs)

# 4. Run optimize function
pareto_result = pareto_optimizer.optimize(pareto_fronts="auto", min_candidates=3)


In [ ]:

# 5. Check results
print("Pareto Optimization Results:")
print(f"Number of Pareto solutions: {len(pareto_result.pareto_solutions)}")
print(f"Number of Pareto fronts: {pareto_result.pareto_fronts}")
print("\nPareto-optimal solutions:")
print(pareto_result.result_hyp_param[['solID', 'nrmse', 'decomp.rssd', 'mape', 'robynPareto']])

print("\nAggregated decomposition results:")
print(pareto_result.x_decomp_agg.head())

print("\nMedia vector collection:")
print(pareto_result.media_vec_collect.head())

print("\nDecomposition vector collection:")
print(pareto_result.x_decomp_vec_collect.head())

# 6. Validate logic
assert len(pareto_result.pareto_solutions) > 0, "No Pareto-optimal solutions found"
assert pareto_result.pareto_fronts > 0, "Invalid number of Pareto fronts"
assert not pareto_result.result_hyp_param.empty, "Empty result_hyp_param DataFrame"
assert not pareto_result.x_decomp_agg.empty, "Empty x_decomp_agg DataFrame"
assert not pareto_result.media_vec_collect.empty, "Empty media_vec_collect DataFrame"
assert not pareto_result.x_decomp_vec_collect.empty, "Empty x_decomp_vec_collect DataFrame"

print("\nAll assertions passed. The optimize function is working as expected.")